In [ ]:
!pip install simpletransformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
fold = 0

In [ ]:
df = pd.read_csv('/content/drive/My Drive/FNEWS/train_folds.csv')
train_df = df.loc[df.kfold != fold]
val_df = df.loc[df.kfold == fold]
train_df.label = train_df.label.apply(lambda x: 1 if x=='fake' else 0)
val_df.label = val_df.label.apply(lambda x: 1 if x=='fake' else 0)
train_df = train_df.drop(columns=['kfold'])
val_df = val_df.drop(columns=['kfold'])

In [ ]:
train_df.shape

In [ ]:
!rm -rf outputs/

In [ ]:
model = None
import gc
gc.collect()

In [ ]:
# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base',
                            args= {'train_batch_size': 64, 'eval_batch_size': 64, 'fp16': True, 'overwrite_output_dir': True}) # You can set class weights by using the optional weight argument

# Train the model
model.train_model(train_df)

Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(val_df)

In [ ]:
test_df = pd.read_csv('/content/drive/My Drive/FNEWS/test.csv')
test = test_df.text.tolist()
predictions0, raw_outputs0 = model.predict(test)

In [ ]:
type(raw_outputs0)

In [ ]:
import numpy as np
with open('roberta-preds01234.npy', 'wb') as f:
  np.save(f, raw_outputs0)
  np.save(f, raw_outputs1)
  np.save(f, raw_outputs2)
  np.save(f, raw_outputs3)
  np.save(f, raw_outputs4)
!cp roberta-preds01234.npy "/content/drive/My Drive/FNEWS/"

In [ ]:
mean_pred = raw_outputs0 + raw_outputs1 + raw_outputs2 + raw_outputs3 + raw_outputs4
mean_pred

In [ ]:
final_pred = [i.argmax() for i in mean_pred]

In [ ]:
submission = pd.DataFrame({'label': final_pred})
submission.head()

In [ ]:
submission.label = submission.label.apply(lambda x: "fake" if x == 1 else "real")

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission_roberta.csv', index=False)
!cp submission_roberta_3fold.csv "/content/drive/My Drive/FNEWS/"